In [1]:
import moviepy.editor as mpe
import numpy as np

video = mpe.VideoFileClip("VideoLeitura.mp4")

In [2]:
def invert_axis(get_frame, t):
    frame = get_frame(t)
    if (int(t) // 20) % 2 == 1:
        return frame[:, ::-1]
    return frame

video_with_inversion = video.fl(invert_axis)


In [3]:
cut_start = 60
cut_end = 80

first = video_with_inversion.subclip(0, cut_start)
middle = video_with_inversion.subclip(cut_start, cut_end)
end = video_with_inversion.subclip(cut_end, video.duration)

video_with_inversion = mpe.concatenate_videoclips([first, end, middle])

In [4]:
audio = video_with_inversion.audio
def decrease_volume(get_frame, t):
    frame = get_frame(t)
    if isinstance(t, (int, float)):
        if t >= (video_with_inversion.duration - 10):
            _fade = 0
        else:
            _fade = max(0, 1 - (t // 30) * 0.1)
    else:
        _fade = 1 - (t // 30) * 0.2
        _fade = np.clip(_fade, 0, 1)

    if len(frame.shape) > 1:
        _fade = _fade[:, np.newaxis]
    
    return frame * _fade

def mute(get_frame, t):
    frame = get_frame(t)
    if isinstance(t, (int, float)) and t >= (video_with_inversion.duration - 10):
        return frame * 0
    elif isinstance(t, np.ndarray):
        mute_factor = np.where(t >= (video_with_inversion.duration - 10), 0, 1)
        if len(frame.shape) > 1:
            mute_factor = mute_factor[:, np.newaxis]
        return frame * mute_factor
    return frame

adjusted_audio = audio.fl(decrease_volume).fl(mute)
video_with_inversion = video_with_inversion.set_audio(adjusted_audio)

In [5]:
video_with_inversion.ipython_display(width=480, maxduration=140)
#save the video
video_with_inversion.write_videofile("Resultado.mp4", codec="libx264", audio_codec="aac")

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


chunk:   0%|          | 0/2956 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
Moviepy - Building video Resultado.mp4.
MoviePy - Writing audio in ResultadoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video Resultado.mp4



Moviepy - Done !
Moviepy - video ready Resultado.mp4
